# Data Preprocessing

After exploring the data, I made notes of the steps we need to take to prepare the data:

* Drop `quantity_group`, `wpt_name`, `recorded_by`, `source_type`, `waterpoint_type_group`, `payment_type` columns
* Replace missing values with `other` category in `funder`, `installer`, `subvillage`, `scheme_name` columns.
* Replace missing values in `public_meeting`, `permit` columns with `True`
* Replace missing values in `scheme_management` to `VWC`
* Replace zeros in `construction_year` with average 
* Use frequency distribution for values in these columns: `funder`, `installer`, `subvillage`, `scheme_name`, `ward`, `lga`, `construction_year`.
* One-hot Encode `scheme_management`, `basin`, `region`,`region_code`, `district_code`, `extraction_type`, `extraction_type_group`, `extraction_type_class`, `management`, `management_group`, `payment`, `water_quality`, `quality_group`, `source`, `source_class`, `waterpoint_type`, `public_meeting`, `permit`, `quantity`.
* Normalize `amount_tsh`, `gps_height`, `population`, `num_private`, `construction_year` columns
* Create features: amount_tsh: `gps_height ratio`, `years_usage`, `water_supply_status`
* Handle imbalanced class labels
* Split data - also drop `id` and `data_recorded` columns

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 200

# load transformed data
data = pd.read_csv('../data/interim/transformed_data.csv')

C:\Users\Home\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Drop Columns

In [2]:
# a list of columns to drop
cols_to_drop = ['quantity_group', 'wpt_name', 'recorded_by', 'source_type', 'waterpoint_type_group', 'payment_type']

In [58]:
def drop_columns(df, cols):
    """Drops columns
    
    input: 1). A Pandas Dataframe and 2). A list of strings
    output: A Pandas Dataframe without the dropped columns
    """
    df = df.copy()
    df = df.drop(cols, axis = 1)
    return df

transformed_data = drop_columns(data, cols_to_drop)
print("Columns dropped! Dataframe now contains {} columns".format(transformed_data.shape[1]))
transformed_data.head(1)

Columns dropped! Dataframe now contains 35 columns


,amount_tsh,basin,construction_year,date_recorded,district_code,extraction_type,extraction_type_class,extraction_type_group,funder,gps_height,id,installer,latitude,lga,longitude,management,management_group,num_private,payment,permit,population,public_meeting,quality_group,quantity,region,region_code,scheme_management,scheme_name,source,source_class,status_group,subvillage,ward,water_quality,waterpoint_type
0,6000.0,Lake Nyasa,1999,2011-03-14,5,gravity,gravity,gravity,Roman,1390,69572,Roman,-9.856322,Ludewa,34.938093,vwc,user-group,0,pay annually,False,109,True,good,enough,Iringa,11,VWC,Roman,spring,groundwater,functional,Mnyusi B,Mundindi,soft,communal standpipe


## Replace Missing Values

The missing values need to be replaced with some specified value.  The value will vary per column.  

Missing values in these columns will be replaced by a random integer:

* **`funder`**
* **`installer`**
* **`subvillage`**
* **`scheme_name`**

Missing values in these columns will be replaced by a string:

* **`public_meeting`** - replace with 'True'
* **`permit`** - replace with 'True'
* **`scheme_management`** - replace with 'VWC'

In [59]:
def fill_missing_vals(df, col, value):
    """Replace missing value with a scalar
    
    input: 1). A Pandas Dataframe, 2). Column, 3). Value to fill holes: scalar
    output: A Pandas Dataframe with no missing values
    """
    df = df.copy()
    return df[col].fillna(value = value)

In [60]:
# a list of a few columns containing missing values.  
missing_val_cols = ['funder', 'installer', 'subvillage', 'scheme_name']

# Replace missing values with this number.  I used an arbitrary value that isn't currently found in the columns. 
filler = 4000

for col in missing_val_cols:
    transformed_data[col] = fill_missing_vals(transformed_data, col, value = filler)
    print('{} now contains {} missing values'.format(col, transformed_data[col].isnull().sum()))

funder now contains 0 missing values
installer now contains 0 missing values
subvillage now contains 0 missing values
scheme_name now contains 0 missing values


In [61]:
missing_val_cols = ['public_meeting', 'permit']

# for this set of columns, replace missing values with 'True'
filler = 'True'

for col in missing_val_cols:
    transformed_data[col] = fill_missing_vals(transformed_data, col, value = filler)
    print('{} now contains {} missing values'.format(col, transformed_data[col].isnull().sum()))

public_meeting now contains 0 missing values
permit now contains 0 missing values


In [62]:
col = 'scheme_management'
filler = 'VWC'

transformed_data[col] = fill_missing_vals(transformed_data, col, value = filler)
print('{} now contains {} missing values'.format(col, transformed_data[col].isnull().sum()))

scheme_management now contains 0 missing values


In [63]:
# There are missing values in the `status_group` column.
print('There are {} missing values'.format(transformed_data.isnull().sum().sum()))

There are 14850 missing values


## Replace Other Values

I need to replace zeros in the **`construction_year`** column with 1997.  This is the average of the column without zeros.  We calculated this value in the data exploration notebook.  

In [64]:
avg_year = 1997
col = 'construction_year'
transformed_data[col] = transformed_data[col].replace(to_replace = 0, value = 1997)
print("The average of the construction year column is: ", round(transformed_data[col].mean()))

The average of the construction year column is:  1997


Now that we have filled the missing values in the `construction_year` column, we have a better idea of the frequency distribution of the column.  We are also able to compare the year the wells were constructed to the year the data about the well was recorded.  

## Handle Categorical Variables

### Use Frequency Distribution

Perform action on the following columns:
* funder
* installer
* subvillage
* scheme_name
* ward
* lga

These columns contained too many unique values to be One-Hot encoded. Instead, I'll replace the text values with their respective frequency. 

In [65]:
columns = ['funder', 'installer', 'subvillage', 'scheme_name', 'ward', 'lga']

def replace(x, counts):
    """Replaces the current value x with the frequency of that unique value"""
    return counts[x]

for col in columns:
    # store frequency of each unique value in a dictionary
    counts = dict(transformed_data[col].value_counts(normalize = True))
    
    # Apply a replace function to every element in the column
    transformed_data[col] = transformed_data[col].apply(lambda x: replace(x, counts))
    print('{} transformed'.format(col))

# Sample the relevant columns
sample_data = transformed_data[columns]
print("\n")
print(sample_data.head())

funder transformed
installer transformed
subvillage transformed
scheme_name transformed
ward transformed
lga transformed


     funder  installer  subvillage  scheme_name      ward       lga
0  0.004525   0.001603    0.000081     0.002303  0.000552  0.009253
1  0.000862   0.000673    0.000081     0.474855  0.001212  0.012189
2  0.000121   0.006963    0.008498     0.000135  0.000175  0.005010
3  0.017832   0.003838    0.000323     0.474855  0.000552  0.002869
4  0.000013   0.002276    0.000027     0.474855  0.000189  0.013024


From the sample, you can see that the frequencies of each unique value replaced the initial string values.  

### One-Hot Encoding 

Most of our categorical variables can afford to be encoded using One-Hot encoding.  I created a list below:

In [66]:
# Perform one-hot encoding on a list of columns
columns = ['scheme_management', 'basin', 'region', 'region_code', 'district_code', 
           'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 
           'management_group', 'payment', 'water_quality', 'quality_group', 
           'source', 'source_class', 'waterpoint_type', 'public_meeting', 'permit', 'quantity']

transformed_data = pd.get_dummies(transformed_data, columns = columns)
print('dummies created!')
print('Dataframe shape:', transformed_data.shape)

dummies created!
Dataframe shape: (74250, 212)


In [67]:
transformed_data.head(2)

,amount_tsh,construction_year,date_recorded,funder,gps_height,id,installer,latitude,lga,longitude,num_private,population,scheme_name,status_group,subvillage,ward,scheme_management_Company,scheme_management_None,scheme_management_Other,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,region_Arusha,region_Dar es Salaam,region_Dodoma,region_Iringa,region_Kagera,region_Kigoma,region_Kilimanjaro,region_Lindi,region_Manyara,region_Mara,region_Mbeya,region_Morogoro,region_Mtwara,region_Mwanza,region_Pwani,region_Rukwa,region_Ruvuma,region_Shinyanga,region_Singida,region_Tabora,region_Tanga,region_code_1,region_code_2,region_code_3,region_code_4,region_code_5,region_code_6,region_code_7,region_code_8,region_code_9,region_code_10,region_code_11,region_code_12,region_code_13,region_code_14,region_code_15,region_code_16,region_code_17,region_code_18,region_code_19,region_code_20,region_code_21,region_code_24,region_code_40,region_code_60,region_code_80,region_code_90,region_code_99,district_code_0,district_code_1,district_code_2,district_code_3,district_code_4,district_code_5,district_code_6,district_code_7,district_code_8,district_code_13,district_code_23,district_code_30,district_code_33,district_code_43,district_code_53,district_code_60,district_code_62,...,extraction_type_india mark iii,extraction_type_ksb,extraction_type_mono,extraction_type_nira/tanira,extraction_type_other,extraction_type_other - mkulima/shinyanga,extraction_type_other - play pump,extraction_type_other - rope pump,extraction_type_other - swn 81,extraction_type_submersible,extraction_type_swn 80,extraction_type_walimi,extraction_type_windmill,extraction_type_group_afridev,extraction_type_group_gravity,extraction_type_group_india mark ii,extraction_type_group_india mark iii,extraction_type_group_mono,extraction_type_group_nira/tanira,extraction_type_group_other,extraction_type_group_other handpump,extraction_type_group_other motorpump,extraction_type_group_rope pump,extraction_type_group_submersible,extraction_type_group_swn 80,extraction_type_group_wind-powered,extraction_type_class_gravity,extraction_type_class_handpump,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,management_company,management_other,management_other - school,management_parastatal,management_private operator,management_trust,management_unknown,management_vwc,management_water authority,management_water board,management_wua,management_wug,management_group_commercial,management_group_other,management_group_parastatal,management_group_unknown,management_group_user-group,payment_never pay,payment_other,payment_pay annually,payment_pay monthly,payment_pay per bucket,payment_pay when scheme fails,payment_unknown,water_quality_coloured,water_quality_fluoride,water_quality_fluoride abandoned,water_quality_milky,water_quality_salty,water_quality_salty abandoned,water_quality_soft,water_quality_unknown,quality_group_colored,quality_group_fluoride,quality_group_good,quality_group_milky,quality_group_salty,quality_group_unknown,source_dam,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,public_meeting_False,public_meeting_True,public_meeting_True,permit_False,permit_True,permit

## Handle Continuous Variables

### Normalize columns:

* amount_tsh
* gps_height
* population
* num_private

In [68]:
from sklearn.preprocessing import MinMaxScaler

def normalize(df, col):
    """Normalizes a column"""
    X = df[col].reshape(-1, 1)
    mms = MinMaxScaler().fit(X = X)
    return mms.transform(X = X)

cont_cols = ['amount_tsh', 'gps_height', 'population', 'num_private']

for col in cont_cols:
    transformed_data[col] = normalize(transformed_data, col)

# Sample 
sample_cont_data = transformed_data[cont_cols]
print(sample_cont_data.head())

C:\Users\Home\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Home\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Home\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


   amount_tsh  gps_height  population  num_private
0    0.017143    0.516219    0.003574          0.0
1    0.000000    0.519358    0.009180          0.0
2    0.000071    0.270666    0.008197          0.0
3    0.000000    0.123125    0.001902          0.0
4    0.000000    0.031392    0.000000          0.0


## Create Additional Features

In [69]:
pd.options.display.max_columns = 205
transformed_data.head()

,amount_tsh,construction_year,date_recorded,funder,gps_height,id,installer,latitude,lga,longitude,num_private,population,scheme_name,status_group,subvillage,ward,scheme_management_Company,scheme_management_None,scheme_management_Other,scheme_management_Parastatal,scheme_management_Private operator,scheme_management_SWC,scheme_management_Trust,scheme_management_VWC,scheme_management_WUA,scheme_management_WUG,scheme_management_Water Board,scheme_management_Water authority,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,region_Arusha,region_Dar es Salaam,region_Dodoma,region_Iringa,region_Kagera,region_Kigoma,region_Kilimanjaro,region_Lindi,region_Manyara,region_Mara,region_Mbeya,region_Morogoro,region_Mtwara,region_Mwanza,region_Pwani,region_Rukwa,region_Ruvuma,region_Shinyanga,region_Singida,region_Tabora,region_Tanga,region_code_1,region_code_2,region_code_3,region_code_4,region_code_5,region_code_6,region_code_7,region_code_8,region_code_9,region_code_10,region_code_11,region_code_12,region_code_13,region_code_14,region_code_15,region_code_16,region_code_17,region_code_18,region_code_19,region_code_20,region_code_21,region_code_24,region_code_40,region_code_60,region_code_80,region_code_90,region_code_99,district_code_0,district_code_1,district_code_2,district_code_3,district_code_4,district_code_5,district_code_6,district_code_7,district_code_8,district_code_13,district_code_23,district_code_30,district_code_33,district_code_43,district_code_53,district_code_60,district_code_62,...,extraction_type_india mark iii,extraction_type_ksb,extraction_type_mono,extraction_type_nira/tanira,extraction_type_other,extraction_type_other - mkulima/shinyanga,extraction_type_other - play pump,extraction_type_other - rope pump,extraction_type_other - swn 81,extraction_type_submersible,extraction_type_swn 80,extraction_type_walimi,extraction_type_windmill,extraction_type_group_afridev,extraction_type_group_gravity,extraction_type_group_india mark ii,extraction_type_group_india mark iii,extraction_type_group_mono,extraction_type_group_nira/tanira,extraction_type_group_other,extraction_type_group_other handpump,extraction_type_group_other motorpump,extraction_type_group_rope pump,extraction_type_group_submersible,extraction_type_group_swn 80,extraction_type_group_wind-powered,extraction_type_class_gravity,extraction_type_class_handpump,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,management_company,management_other,management_other - school,management_parastatal,management_private operator,management_trust,management_unknown,management_vwc,management_water authority,management_water board,management_wua,management_wug,management_group_commercial,management_group_other,management_group_parastatal,management_group_unknown,management_group_user-group,payment_never pay,payment_other,payment_pay annually,payment_pay monthly,payment_pay per bucket,payment_pay when scheme fails,payment_unknown,water_quality_coloured,water_quality_fluoride,water_quality_fluoride abandoned,water_quality_milky,water_quality_salty,water_quality_salty abandoned,water_quality_soft,water_quality_unknown,quality_group_colored,quality_group_fluoride,quality_group_good,quality_group_milky,quality_group_salty,quality_group_unknown,source_dam,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,public_meeting_False,public_meeting_True,public_meeting_True,permit_False,permit_True,permit

Here's a list of additional features that we can create:

* amount_tsh:gps_height ratio
* age (the date the well was created to the date the data was recorded)
* water supply status

### amount_tsh:gps_height

In [70]:
transformed_data['amount_tsh_gps_height'] = transformed_data['amount_tsh'] / transformed_data['gps_height']

### age


In [71]:
# convert year_recorded column to datetime
transformed_data['date_recorded'] = pd.to_datetime(transformed_data['date_recorded'])

# create column containing the year only
transformed_data['year_recorded'] = pd.DatetimeIndex(transformed_data['date_recorded']).year

# calc age of wells
transformed_data['well_age'] = transformed_data['year_recorded'] - transformed_data['construction_year']
print(transformed_data['well_age'].describe())

count    74250.000000
mean        15.002478
std         10.067292
min         -7.000000
25%          8.000000
50%         14.000000
75%         16.000000
max         53.000000
Name: well_age, dtype: float64


Notice that the min value is negative.  This implies that the data for a few samples were collected before the well was built. Let's take a further look into these samples.  

In [72]:
# focus on these columns only
cols = ['construction_year', 'well_age', 'year_recorded', 'date_recorded']
filtered_data = transformed_data[cols]

# filter for samples whose age is less than 0
odd_wells = filtered_data[filtered_data.well_age < 0]
print(odd_wells)

       construction_year  well_age  year_recorded date_recorded
8729                2008        -4           2004    2004-05-01
10441               2006        -2           2004    2004-12-01
13366               2011        -7           2004    2004-06-01
23373               2009        -5           2004    2004-03-01
27501               2009        -5           2004    2004-01-07
32619               2005        -1           2004    2004-03-01
33942               2007        -3           2004    2004-03-01
39559               2009        -5           2004    2004-03-06
48555               2008        -4           2004    2004-12-01
62322               2008        -4           2004    2004-01-04
66450               2011        -7           2004    2004-06-01
67118               2006        -2           2004    2004-06-01


We can see that the data for these rows were collected in 2004.  For these rows, I will replace the values in the `well_age` column with zeros

In [73]:
def replace_age(x):
    """Replace negative values with zeros"""
    if x < 0:
        x = 0
    return x

transformed_data['well_age'] = transformed_data['well_age'].apply(lambda x: replace_age(x))

# normalize columns
transformed_data['well_age'] = normalize(transformed_data, 'well_age')
transformed_data['construction_year'] = normalize(transformed_data, 'construction_year')
transformed_data['year_recorded'] = normalize(transformed_data, 'year_recorded')

transformed_data['well_age'].describe()

C:\Users\Home\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Home\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Home\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


count    74250.000000
mean         0.283078
std          0.189927
min          0.000000
25%          0.150943
50%          0.264151
75%          0.301887
max          1.000000
Name: well_age, dtype: float64

After applying the transformation, we see that the min value is now zero.  

### water supply status

Here we will create a feature called `water_supply_status`.  There are a few conditions to consider:

1. Inhabited areas without water
2. Inhabited areas with water
3. Uninhabited areas without water
4. Uninhabited areas with water

I assigned a unique value for each condition below.  

In [74]:
no_water_people = transformed_data[(transformed_data.population > 0.0) & (transformed_data.amount_tsh == 0.0)].index
water_people = transformed_data[(transformed_data.population > 0.0) & (transformed_data.amount_tsh > 0.0)].index
no_water_no_people = transformed_data[(transformed_data.population == 0.0) & (transformed_data.amount_tsh == 0.0)].index
water_no_people = transformed_data[(transformed_data.population == 0.0) & (transformed_data.amount_tsh > 0.0)].index

# assign unique value for each condition
transformed_data.loc[no_water_people, 'water_supply_status'] = 0
transformed_data.loc[water_people, 'water_supply_status'] = 1
transformed_data.loc[no_water_no_people, 'water_supply_status'] = 2
transformed_data.loc[water_no_people, 'water_supply_status'] = 3

# normalize column
transformed_data['water_supply_status'] = normalize(transformed_data, 'water_supply_status')
transformed_data[['water_supply_status']].head()

,water_supply_status
0,0.333333
1,0.000000
2,0.333333
3,0.000000
4,0.666667


In [75]:
# drop id and date_recorded columns
cols = ['id', 'date_recorded']
transformed_data = drop_columns(transformed_data, cols)

## Partition Data

In [76]:
# split into training and test sets
train = transformed_data[transformed_data['status_group'].notnull()]
test = transformed_data[transformed_data['status_group'].isnull()].drop(['status_group'], axis = 1)
print('Training set contains {} rows and {} columns'.format(train.shape[0], train.shape[1]))
print('Test set contains {} rows and {} columns'.format(test.shape[0], test.shape[1]))

Training set contains 59400 rows and 214 columns
Test set contains 14850 rows and 213 columns


## Encoding class label

In [78]:
from sklearn.preprocessing import LabelEncoder

def label_encoder(df, col):
    class_le = LabelEncoder()
    # apply label encoder on the `status_group` column
    y = class_le.fit_transform(df[col])
    df.loc[:, col] = y
    return df[col]

train.loc[:, 'status_group'] = label_encoder(train, 'status_group')

# Sample of class label column
train[['status_group']][:5]

C:\Users\Home\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,status_group
0,0
1,0
2,0
3,2
4,0


Now that we have encoded the `status_group` column, the data preprocessing step is complete.  Let's save this dataframe.

In [84]:
# Save progress
train.to_csv('../data/processed/train.csv', index = False)
test.to_csv('../data/processed/test.csv', index = False)